In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Pragyasharava\\Desktop\\leukemia-detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Pragyasharava\\Desktop\\leukemia-detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config
    

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D

In [ ]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        # Ensure image_size is a list
        image_size = list(self.config.params_image_size)

        
        weights = self.config.params_weights
        if isinstance(weights, tf.Tensor):
            weights = weights.numpy().decode('utf-8')  # Convert tensor to string if needed

        # Print the values to verify
        print("Image Size:", image_size)
        print("Weights:", weights)

        # Create the model
        self.model = tf.keras.applications.EfficientNetB3(
            input_shape=image_size,
            weights=weights,  
            include_top=self.config.params_include_top
        )



        # Save the model 
        self.model.save(self.config.base_model_path)  





    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False


        # Add custom classification layers
        flatten_in = GlobalAveragePooling2D()(model.output)
        dense_1 = Dense(128, activation='relu')(flatten_in)
        dropout = Dropout(0.6)(dense_1) # Regularization
        prediction = Dense(classes, activation="softmax")(dropout)
        

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)

        # Compile the model
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=False,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        # Save the model in TensorFlow SavedModel format
        self.full_model.save(self.config.updated_base_model_path, save_format='tf')



    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

        



In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-02-18 18:19:01,333: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-18 18:19:01,336: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-18 18:19:01,338: INFO: common: created directory at: artifacts]
[2025-02-18 18:19:01,339: INFO: common: created directory at: artifacts/prepare_base_model]
Image Size: [224, 224, 3]
Weights: imagenet
[2025-02-18 18:19:03,982: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2025-02-18 18:19:34,447: INFO: builder_impl: Assets written to: artifacts\prepare_base_model\base_model\assets]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                   